<a href="https://colab.research.google.com/github/Mystique970/SummerAnalytics/blob/main/SA_CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Dynamic Pricing Models for Urban Parking Lots

This notebook implements three models:
- **Model 1:** Baseline Linear Pricing
- **Model 2:** Demand-Based Pricing
- **Model 3:** Competitive Pricing with Location Intelligence

Each model updates prices in real-time using Pathway streaming and Bokeh for visualization.

Installing the Libraries

---



In [56]:
!pip install pathway bokeh --quiet

# **Dynamic Pricing for Urban Parking Lots**

Importing All the Required Libraries

---



In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# **Model 1: Baseline Linear Model**

Loading the Dataset

---



In [58]:
from google.colab import files
uploaded = files.upload()


Saving dataset.csv to dataset (2).csv


# **Model 2: Demand-Based Price Function**

In [59]:
df = pd.read_csv('dataset.csv')

In [60]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


Creating and Sorting the Timestamp Column

---



In [61]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

In [62]:
from sklearn.preprocessing import OrdinalEncoder

vehicle_order = [['cycle','bike', 'car', 'truck']]

encoder = OrdinalEncoder(categories=vehicle_order)

df['VehicleType_ordinal'] = encoder.fit_transform(df[['VehicleType']])

df['VehicleType_ordinal'] = df['VehicleType_ordinal'].astype(int)

In [63]:
traffic_order = [['low','average', 'high']]

encoder = OrdinalEncoder(categories=traffic_order)

df['TrafficConditionNearby_ordinal'] = encoder.fit_transform(df[['TrafficConditionNearby']])

df['TrafficConditionNearby_ordinal'] = df['TrafficConditionNearby_ordinal'].astype(int)

Saving Selected Columns to a New CSV File

---



In [64]:
df[["Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby_ordinal", "IsSpecialDay", "VehicleType_ordinal"]].to_csv("parking_stream2.csv", index=False)

Defining the Extended Schema for the New CSV File

---



In [65]:
class ParkingSchema2(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby_ordinal: int
    IsSpecialDay: int
    VehicleType_ordinal: int

In [66]:
data = pw.demo.replay_csv("parking_stream2.csv", schema=ParkingSchema2, input_rate=1000)

In [67]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [68]:
def bound_price(price):
    if price < 5:
        return 5
    elif price > 20:
        return 20
    else:
        return price

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_level=pw.reducers.max(pw.this.TrafficConditionNearby_ordinal),
        special_day=pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_type_weight=pw.reducers.max(pw.this.VehicleType_ordinal),
    )
    .with_columns(
        demand = 1.5 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap) ** 2 +
                 1 * pw.this.queue_length -
                 0.5 * (pw.this.traffic_level) ** 0.5 +
                 0.4 * pw.this.special_day +
                 0.6 * pw.this.vehicle_type_weight,
    )
    .with_columns(
        normalized_demand = (pw.this.demand - 0) / (10 - 0),
    )
    .with_columns(
        price_raw = 10 * (1 + 0.5 * pw.this.normalized_demand),
    )
    .with_columns(
        price = pw.apply(bound_price, pw.this.price_raw)
)
)

Encoding Vehicle Types Using Ordinal Encoding

---



In [69]:
from bokeh.io import output_notebook
output_notebook()


In [70]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")

pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [79]:
%%capture --no-display
pw.run()

Output()

In [80]:
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

In [81]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

Loading the Data into Pathway as a Stream

---



In [82]:
data = pw.demo.replay_csv("parking_stream2.csv", schema=ParkingSchema2, input_rate=1000)

Converting Timestamp to Datetime and Creating a Day Column

---



In [83]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

Creating a Windowed Table with Daily Max/Min Occupancy

---



**Formula used for this model**                                 
Price(t+1) = Price(t) + 0.1 * (Occupancy(t+1) - Occupancy(t)) / (Occupancy_max - Occupancy_min)

In [84]:
windowed = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        day=pw.reducers.min(pw.this.day),
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
    )
)


joined = data_with_time.join(
    windowed,
    data_with_time.day == windowed.day
).select(
    t = pw.left.t,
    day = pw.left.day,
    Occupancy = pw.left.Occupancy,
    Capacity = pw.left.Capacity,
    occ_max = pw.right.occ_max,
    occ_min = pw.right.occ_min,
)

joined = joined.with_columns(
    lag_key = pw.make_tuple(pw.this.day, pw.this.t),
    lag_key_prev = pw.make_tuple(pw.this.day, pw.this.t - datetime.timedelta(hours=1))
)

left = joined
right = joined.copy()


lagged = left.join(
    right,
    left.lag_key_prev == right.lag_key,
    how=pw.JoinMode.LEFT
).select(
    t = left.t,
    day = left.day,
    Occupancy = left.Occupancy,
    Capacity = left.Capacity,
    occ_max = left.occ_max,
    occ_min = left.occ_min,
    Occupancy_prev = pw.if_else(
        right.Occupancy.is_not_none(),
        right.Occupancy,
        left.Occupancy
    )
)

result = lagged.with_columns(
    Price = pw.if_else(
        (lagged.occ_max - lagged.occ_min) != 0,
        lagged.Occupancy_prev + 0.1 * (lagged.Occupancy - lagged.Occupancy_prev) / (lagged.occ_max - lagged.occ_min),
        lagged.Occupancy
    )
)


In [85]:
BASE_PRICE = 10
alpha = 2

model1 = lagged.with_columns(
    Price_Model1 = pw.if_else(
        lagged.Capacity != 0,
        BASE_PRICE + alpha * (lagged.Occupancy / lagged.Capacity),
        BASE_PRICE
    )
)


Calculating Dynamic Price Based on Demand Factors

---



**Formula for Demand:**                                   
Demand = 1.5 * ((Occupancy_max - Occupancy_min) / Capacity)^2
         + 1 * QueueLength
         - 0.5 * sqrt(TrafficLevel)
         + 0.4 * IsSpecialDay
         + 0.6 * VehicleType

**Normalize the demand:**        
Normalized Demand = (Demand - 0) / (10 - 0)

**Calculate raw price:**      
RawPrice = 10 * (1 + 0.5 * NormalizedDemand)

**Apply price bounds:**   
Final Price = min(20, max(5, RawPrice))

In [90]:
alpha, beta, gamma, delta, epsilon, lambd = 0.4, 0.3, 0.2, 0.2, 0.2, 0.3

model2 = data_with_time.with_columns(
    Demand = (
        alpha * (data_with_time.Occupancy / data_with_time.Capacity)
        + beta * data_with_time.QueueLength
        - gamma * data_with_time.TrafficConditionNearby_ordinal
        + delta * data_with_time.IsSpecialDay
        + epsilon * data_with_time.VehicleType_ordinal
    )
)

model2.select(
    Timestamp = data_with_time.Timestamp,

)

<pathway.Table schema={'Timestamp': <class 'str'>}>

Visualizing the Final Dynamic Price Output

---



In [91]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "Price", source=source, line_width=2, color="navy")
    fig.circle("t", "Price", source=source, size=6, color="red")
    return fig

viz = result.plot(price_plotter, sorting_col="t")

pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

Running the Pathway Pipeline

---



In [ ]:
%%capture --no-display
pw.run()

Output()